In [2]:
import pandas as pd
import numpy as np
import plotly.express as px

In [3]:
df = pd.DataFrame(pd.read_csv("../assets/normalized_pyramid_data.csv"))

In [4]:
key = ['unknown', 'pyramid?']
complexes = df[~df['pyramid_complex'].isin(key)]
kings_and_queens = complexes.groupby(['pyramid_complex', 'royal_status'], sort=False).size().reset_index(name='count')

confirmed_count_fig = px.bar(kings_and_queens, 
             x='pyramid_complex', 
             y='count', 
             color='royal_status',
             title='Number of Confirmed Pyramids At Each Complex',
             labels={
                 'pyramid_complex': 'Pyramid Complex',
                 'count': 'Number of Pyramids',
                 'royal_status': 'Royal Status'
             },
             custom_data=['royal_status'])
confirmed_count_fig.update_traces(
    hovertemplate='<br>'.join([
        'Royal status: %{customdata[0]}',
        'Pyramid Complex: %{x}',
        'Total: %{y}',
        '<extra></extra>'
    ])
)
confirmed_count_fig.show()
df

,pyramid_owner,pyramid_complex,royal_status,daughter_of,royal_mother_title,likely_wife,wife_title,vizier,regent,relationship_to_king,...,site_or_location,orientation,casing,state_of_completion,superstructure_type,height,length,width,angle,notes
0,Djoser,Djoser,King,NaN,False,False,False,False,False,NaN,...,Saqqara,N-S,Limestone,NaN,Pyramid,59.9,120.00,108.00,74,"15 gates, white limestone casing"
1,Sekhemkhet,Sekhemkhet,King,NaN,False,False,False,False,False,NaN,...,Saqqara,N-S,Limestone,Unfinished,Pyramid,70,120.00,120.00,71-75,"niched facade, unfinished, left with rough ext..."
2,Nebka?,Nebka?,King,NaN,False,False,False,False,False,NaN,...,Zawiyet el-Aryan,E-W,Unknown,Unfinished,Pyramid,NaN,200.00,200.00,unknown,debate over whether this is Pyramid of Nebka
3,Khaba,Khaba,King,NaN,False,False,False,False,False,NaN,...,Zawiyet el-Aryan,E-W,Unknown,Unfinished,Pyramid,42,83.80,83.80,unknown,began as step pyramid
4,Sneferu,Sneferu 1,King,NaN,False,False,False,False,False,NaN,...,Meidum,E-W,Limestone,Unfinished,Pyramid,94.5,144.32,144.32,52,"limestone casing, causeway 200m, Cult Pyramid ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,Neferkaure,unknown,King,NaN,False,False,False,False,False,NaN,...,Unknown,NaN,Unknown,Unknown,Unknown,NaN,NaN,NaN,NaN,NaN
70,Neferkauhor?,pyramid?,King,NaN,False,False,False,False,False,NaN,...,Unknown,NaN,Unknown,Unknown,Unknown,NaN,NaN,NaN,NaN,"no archaeological evidence of a pyramid, but i..."
71,Neferirkare,unknown,King,NaN,False,False,False,False,False,NaN,...,Unknown,NaN,Unknown,Unknown,Unknown,NaN,NaN,NaN,NaN,NaN
72,Wadjkare? (Demedjibtawj),pyramid?,King,NaN,False,False,False,False,False,NaN,...,Unknown,NaN,Unknown,Unknown,Unknown,NaN,NaN,NaN,NaN,"no archaeological evidence of a pyramid, but i..."


In [5]:
dynasty_pyramids = df.groupby(['dynasty', 'royal_status'], sort=False).size().reset_index(name='count')
dynasty_count_fig = px.bar(dynasty_pyramids, 
             x='dynasty', 
             y='count', 
             color='royal_status',
             title='Number of Pyramids Per Dynasty',
             labels={
                 'dynasty': 'Dynasty',
                 'count': 'Number of Pyramids (Including Speculations)',
                 'royal_status': 'Royal Status'
             },
             custom_data=['royal_status'])
dynasty_count_fig.update_traces(
    hovertemplate='<br>'.join([
        'Royal status: %{customdata[0]}',
        'Dynasty: %{x}',
        'Total: %{y}',
        '<extra></extra>'
    ])
)
dynasty_count_fig.show()

In [6]:
# Attempting to recreate George's suggested visualization (picture on my phone)

# Trying to fill 'start_of_reign' fully so that sorts work correctly
# TODO: Add this functionality to the cleanup script
unique_comp = complexes['pyramid_complex'].unique()
temp = df

for comp in unique_comp:
    start = temp[temp['pyramid_complex'] == comp]['start_of_reign'].max()
    temp[temp['pyramid_complex'] == comp]['start_of_reign'] = temp[temp['pyramid_complex'] == comp]['start_of_reign'].replace(np.nan, start)

temp.loc[temp['pyramid_complex'] == 'Sneferu 3', 'start_of_reign'] = 2574   # This had to be done to get it in the correct order (value was missing)
temp.dropna(subset='height', inplace=True)
temp.sort_values(by='start_of_reign', ascending=False, inplace=True)

# Getting the height column to be numeric
def average_of_two(val):
    if isinstance(val, int) or isinstance(val, float) or pd.isna(val): return val

    if ',' in val: return 72    # Temporary: Deals with that one weird value

    nums = val.split('-')
    if len(nums) == 1: return float(nums[0])
    return (float(nums[0]) + float(nums[1])) / 2

temp['height'] = temp['height'].map(average_of_two).astype(float)
tl = temp[['pyramid_complex', 'pyramid_owner', 'start_of_reign', 'end_of_reign', 'length_of_reign', 'height', 'royal_status', 'relationship_to_king']]

# Plotly stuff
grouped_heights_fig = px.bar(tl, 
             x='pyramid_complex', 
             y='height', 
             color='royal_status',
             title='Height of Pyramids At Each Complex By Status',
             labels={
                 'pyramid_complex': 'Pyramid Complex',
                 'height': 'Height (meters)',
                 'royal_status': 'Royal Status'
             },
             barmode='group',
             custom_data=['royal_status', 'pyramid_owner', tl['relationship_to_king'].fillna('Self')])
grouped_heights_fig.update_traces(
    hovertemplate='<br>'.join([
        'Royal status: %{customdata[0]}',
        'Pyramid Complex: %{x}',
        'Height: %{y} m',
        'Pyramid Owner: %{customdata[1]}',
        'Relationship To King: %{customdata[2]}',
        '<extra></extra>'
    ])
)
grouped_heights_fig.show()

/var/folders/q8/nm86scy964318ffbhmh7n8qm0000gr/T/ipykernel_1888/3831433333.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [7]:
complex_heights = tl.groupby('pyramid_complex', sort=False, as_index=False)['height'].sum()

line_heights_fig = px.line(
    complex_heights,
    x='pyramid_complex',
    y='height',
    title='Total Height of All Pyramids At Each Complex',
    labels={
        'pyramid_complex': 'Pyramid Complex',
        'height': 'Aggregated Height (meters)'
    }
)
line_heights_fig.update_traces(
    hovertemplate='<br>'.join([
        'Pyramid Complex: %{x}',
        'Aggregated Height: %{y} m'
    ])
)
line_heights_fig.show()

In [8]:
# Exporting to HTML
from jinja2 import Template

output_html_path = r'../pages/index.html'
input_template_path = r'../templates/template.html'

plotly_jinja_data = {
    'grouped_heights':grouped_heights_fig.to_html(full_html=False),
    'dynasty_count':dynasty_count_fig.to_html(full_html=False),
    'confirmed_count':confirmed_count_fig.to_html(full_html=False),
    'line_heights':line_heights_fig.to_html(full_html=False)
}

with open(output_html_path, 'w', encoding='utf-8') as out:
    with open(input_template_path) as template:
        j2_template = Template(template.read())
        out.write(j2_template.render(plotly_jinja_data))

In [16]:
import plotly.graph_objects as go

# Get the groups with relevant data
com_heights = tl.groupby(['pyramid_complex', 'start_of_reign', 'end_of_reign', 'length_of_reign'], sort=False, as_index=False)['height'].sum()
com_heights['start_of_reign'] = -com_heights['start_of_reign']
com_heights['end_of_reign'] = -com_heights['end_of_reign']

# Select random colors for each complex
# String below copied from the output of an error, which listed valid CSS colors
colors = 'aliceblue, antiquewhite, aqua, aquamarine, azure, beige, bisque, black, blanchedalmond, blue, blueviolet, brown, burlywood, cadetblue, chartreuse, chocolate, coral, cornflowerblue, cornsilk, crimson, cyan, darkblue, darkcyan, darkgoldenrod, darkgray, darkgrey, darkgreen, darkkhaki, darkmagenta, darkolivegreen, darkorange, darkorchid, darkred, darksalmon, darkseagreen, darkslateblue, darkslategray, darkslategrey, darkturquoise, darkviolet, deeppink, deepskyblue, dimgray, dimgrey, dodgerblue, firebrick, floralwhite, forestgreen, fuchsia, gainsboro, ghostwhite, gold, goldenrod, gray, grey, green, greenyellow, honeydew, hotpink, indianred, indigo, ivory, khaki, lavender, lavenderblush, lawngreen, lemonchiffon, lightblue, lightcoral, lightcyan, lightgoldenrodyellow, lightgray, lightgrey, lightgreen, lightpink, lightsalmon, lightseagreen, lightskyblue, lightslategray, lightslategrey, lightsteelblue, lightyellow, lime, limegreen, linen, magenta, maroon, mediumaquamarine, mediumblue, mediumorchid, mediumpurple, mediumseagreen, mediumslateblue, mediumspringgreen, mediumturquoise, mediumvioletred, midnightblue, mintcream, mistyrose, moccasin, navajowhite, navy, oldlace, olive, olivedrab, orange, orangered, orchid, palegoldenrod, palegreen, paleturquoise, palevioletred, papayawhip, peachpuff, peru, pink, plum, powderblue, purple, red, rosybrown, royalblue, rebeccapurple, saddlebrown, salmon, sandybrown, seagreen, seashell, sienna, silver, skyblue, slateblue, slategray, slategrey, snow, springgreen, steelblue, tan, teal, thistle, tomato, turquoise, violet, wheat, white, whitesmoke, yellow, yellowgreen'.split(', ')
choices = np.random.default_rng().choice(len(colors), len(com_heights), replace=False)

# Create figure
figure = go.Figure()
'''
# This adds the bars in a single trace, which doesn't work well with the legend, which
# expects a trace for each bar.
figure.add_trace(
    go.Bar(
        x=com_heights['start_of_reign'],
        y=com_heights['height'],
        width=com_heights['length_of_reign'],
        text=com_heights['pyramid_complex'],
        offset=0.5,
        marker=dict(
            color=choices,
            colorscale=colors
        )
    )
)'''

# Add each bar individually as a trace
for complex, start, end, reign, height in com_heights.values:
    figure.add_trace(
        go.Bar(
            x=[start],
            y=[height],
            width=[reign],
            text=[complex],
            name=complex,
            offset=0.5,
            customdata=[complex, start, end, reign, height],
            hovertemplate='<br>'.join([
                'Pyramid Complex: %{customdata[0]}',
                'Reign: %{customdata[1]} - %{customdata[2]} BCE',
                'Length of Reign: %{customdata[3]} Years',
                'Height: %{customdata[4]} m',
                '<extra></extra>'
            ])
        )
    )
figure.update_xaxes(
    title_text='Time (Years BCE)',
    minor={'showgrid': True, 'dtick': 10, 'tick0': 0.1})
figure.update_yaxes(title_text='Aggregated Height (Meters)')
figure.update_layout(
    autosize=False,
    width=1000,
    height=800,
    title={'text': 'Total Height of All Pyramids At Each Complex'},
    xaxis={
        'tickvals': [i for i in range(-2700, -2100, 25)],
        'tickmode': 'array',
        'showticklabels': True,
        'ticks': 'outside',
        'ticklen': 5
    }
)
figure.show()

# NOTE: Sneferu 3 missing due to absence of start and end values (I think), and 
# not included after the fact because the bar would overlap with Sneferu 2. Figure 
# out a way to resolve this issue (combine all Sneferus?)

In [133]:
queens = temp[temp['royal_status'] == 'Queen']
queen_data = queens[['dynasty', 'royal_status', 'daughter_of', 'royal_mother_title', 'likely_wife', 'wife_title', 'vizier', 'regent', 'relationship_to_king']]
queen_data['dynasty'] = queen_data['dynasty'].astype(int)

by_dyn = queen_data.groupby('dynasty', as_index=False)

'''fig_3d = px.scatter_3d(
    queen_data,
    x='vizier',
    y='likely_wife',
    z='regent',
    color='royal_mother_title'
)'''

# FACET is the word you keep looking for

vizier_dyn = queen_data.groupby('dynasty', as_index=False)['vizier'].value_counts()
fig = px.bar(
    vizier_dyn,
    x='dynasty',
    y='count',
    color='vizier',
    color_discrete_sequence=['red', 'blue'],
    barmode='stack',
    labels={
        'dynasty': 'Dynasty',
        'count': 'Total',
        'vizier': 'Vizier?'
    },
    title='Vizier Queen Pyramids By Dynasty'
)

fig.show()


/var/folders/q8/nm86scy964318ffbhmh7n8qm0000gr/T/ipykernel_1888/1350461184.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [136]:
royal_mother_title_dyn = queen_data.groupby('dynasty', as_index=False)['royal_mother_title'].value_counts()
fig = px.bar(
    royal_mother_title_dyn,
    x='dynasty',
    y='count',
    color='royal_mother_title',
    color_discrete_sequence=['red', 'blue'],
    barmode='stack',
    labels={
        'dynasty': 'Dynasty',
        'count': 'Total',
        'royal_mother_title': 'Royal Mother?'
    },
    title='Royal Mother Queen Pyramids By Dynasty'
)
fig.show()

In [115]:
likely_wife_dyn = queen_data.groupby('dynasty', as_index=False)['likely_wife'].value_counts()
fig = px.bar(
    likely_wife_dyn,
    x='dynasty',
    y='count',
    color='likely_wife',
    barmode='stack',
    labels={
        'dynasty': 'Dynasty',
        'count': 'Total',
        'likely_wife': 'Likely Wife?'
    },
    title='Wife Queen Pyramids By Dynasty'
)
fig.show()



In [135]:
regent_dyn = queen_data.groupby('dynasty', as_index=False)['regent'].value_counts()
fig = px.bar(
    regent_dyn,
    x='dynasty',
    y='count',
    color='regent',
    color_discrete_sequence=['red', 'blue'],
    barmode='stack',
    labels={
        'dynasty': 'Dynasty',
        'count': 'Total',
        'regent': 'Regent?'
    },
    title='Regent Queen Pyramids By Dynasty'
)

fig.show()